In [1]:
import polars as pl 

def mk_stat_polars(y: pl.Series) -> pl.Expr:
    """Apply Mann-Kendall trend test to a Polars column (time series)."""
    n = len(y)
    
    # Initialize variable for sum of ranks (s)
    s = 0
    
    # We will use vectorized operations to compute the sums hopefully 
    for k in range(n-1):
        # Compare each element with all subsequent elements
        greater = (y[k+1:n] > y[k]).sum()
        less = (y[k+1:n] < y[k]).sum()

        s += greater - less
    
    # Mann-Kendall statistic formula: s / (0.5 * n * (n - 1))
    mk_stat = s / (0.5 * n * (n - 1))
    return mk_stat

In [4]:
df = pl.scan_parquet('https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet').collect()


#this is how we currently have to compute these function 


mk_stat_df = df.group_by('unique_id').agg(pl.col('y').map_batches(mk_stat_polars,return_dtype=pl.Float64))

In [6]:
mk_stat_df.explode('y')

unique_id,y
str,f64
"""H280""",0.241971
"""H51""",0.173002
"""H187""",-0.086547
"""H306""",0.362882
"""H139""",0.002319
…,…
"""H384""",0.061882
"""H12""",0.099689
"""H176""",0.44378
